In [7]:
import numpy as np  
import pandas as pd 
from operator import itemgetter
from openpyxl import load_workbook
 

In [8]:
pip install --upgrade prophet

Note: you may need to restart the kernel to use updated packages.


In [9]:
#Prophet 모델

import pandas as pd
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings
import os
import itertools
from prophet.diagnostics import performance_metrics
import joblib
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
#Testing on Cruiser Black - starting date should be 7/30/19

orders_file = "./Order_data/Cruiser/sales_2019-01-01_2022-08-15.csv"
cruiser_df = pd.read_csv(orders_file, keep_default_na=False) #daily order file

#setting up the dates
start_date = '2019-01-01'
end_date = '2022-08-15'

mask = (cruiser_df['day'] > start_date) & (cruiser_df['day'] <= end_date)
cruiser_df=  cruiser_df.loc[mask]

#Order number greater than 0.
cruiser_df = cruiser_df[cruiser_df['net_quantity'] > 0]
# cruiser_df = cruiser_df[cruiser_df['net_quantity'] < 5]

cruiser_df['shipping_country'] == 'Canada'

#North America Order
cruiser_df = cruiser_df[(cruiser_df['shipping_country'] == 'United States') | (cruiser_df['shipping_country'] == 'Canada')| (cruiser_df['shipping_country'].isna())]

cruiser_df = cruiser_df.groupby(by=['day']).sum().groupby(level=[0]).cumsum()
cruiser_df.reset_index(inplace=True)
cruiser_df.rename(columns={"index":"day"}, inplace =True)

cruiser_df['y'] = cruiser_df['net_quantity']
cruiser_df['ds'] = cruiser_df['day']

plot_df = cruiser_df[['ds','y']]
plot_df = plot_df.set_index(['ds'])

ticker = 'cruiser black'
print(plot_df)
ax= plot_df['y'].plot(title = ticker, figsize=(24,16))
ax.set_ylabel('order quantity')

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './Order_data/Cruiser/sales_2019-01-01_2022-08-15.csv'

In [ ]:
##df_final contains input features, demand_df contains full dataset
##Split data in train and test
train_percentage=80
train_final_index=round(len(cruiser_df)*(train_percentage/100))
cruiser_df_train = cruiser_df[0:train_final_index]
cruiser_df_test = cruiser_df[train_final_index:]
# X_train = 
# y_train = 
y_test = cruiser_df[['y']][train_final_index:]

print(cruiser_df_train)
print(cruiser_df_test)

In [ ]:
##Application of Regression model and printing error metrics
# lasso_reg = Lasso(alpha=0.2)
# lasso_reg.fit(X_train, y_train)
# joblib.dump(lasso_reg, "lasso_reg.h5")
# y_pred = lasso_reg.predict(X_test)
# print('RMSE :', np.sqrt(mean_squared_error(y_test, y_pred)))
# print('R2 score - ', r2_score(y_test, y_pred))

In [ ]:
# m = Prophet(weekly_seasonality=False)
# m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
# forecast = m.fit(df).predict(future)
# fig = m.plot_components(forecast)

In [2]:
model = Prophet(changepoint_prior_scale= 0.1, seasonality_prior_scale=2., holidays_prior_scale= 0.01, yearly_seasonality=True, 
        weekly_seasonality=True,
        daily_seasonality=False, 
        interval_width=0.95)
model.add_seasonality(
        name='monthly', 
        period=30.5, 
        fourier_order=5
    )

model.fit(cruiser_df_train)
prediction = model.predict(cruiser_df_test)

# with open('fb_target_model.json', 'w') as fout:
#     json.dump(model_to_json(model), fout)df_future
    
# Plot error metrics
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, prediction['yhat'])))
# print('R2 score - ', r2_score(y_test, prediction['yhat']))
model.plot(prediction)

NameError: name 'Prophet' is not defined

In [160]:
future = model.make_future_dataframe(periods=X_test.shape[0]+10, freq='D', include_history=False)
future_values=future[16:22]
future_values=future_values.reset_index()
future_values.drop(columns='index', inplace=True)
print(future_values)
future_values['Forecasted_Order']=model.predict(future_values)['yhat']
print(future_values)

          ds
0 2022-02-11
1 2022-02-12
2 2022-02-13
3 2022-02-14
4 2022-02-15
5 2022-02-16
          ds  Forecasted_Order
0 2022-02-11          0.920555
1 2022-02-12          0.236517
2 2022-02-13          0.401851
3 2022-02-14          1.165309
4 2022-02-15          1.235307
5 2022-02-16          1.087413


In [153]:
model = Prophet(yearly_seasonality=False, 
        weekly_seasonality=True,
        daily_seasonality=False, 
        interval_width=0.95)
model.add_seasonality(
        name='monthly', 
        period=30.5, 
        fourier_order=5
    )

model.fit(X_train)
prediction = model.predict(X_test)

with open('fb_target_model.json', 'w') as fout:
    json.dump(model_to_json(model), fout)df_future

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10],
    'seasonality_mode':  ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = [] # Store the RMSEs for each params here


SyntaxError: invalid syntax (Temp/ipykernel_17992/2430606561.py, line 15)

In [154]:
pip install dask

In [155]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from dask.distributed import Client

client = Client()
df_cv = cross_validation(m, initial='1000 days', period='28 days', horizon = '7 days', parallel="dask")

C:\Users\hyunk\anaconda3\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57829 instead
  warnings.warn(


In [156]:

# Use cross validation to evaluate all parameters
cutoffs = pd.to_datetime(['2021-02-15', '2021-08-15', '2022-02-15'])

for params in all_params:
    m = Prophet(**params).fit(cruiser_df)  # Fit model with given params
    df_cv = cross_validation(m, cutoffs=cutoffs, horizon='28 days')
    df_p = performance_metrics(df_cv, rolling_window=7)
    rmses.append(df_p['rmse'].values[0])
    
# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

best_params = all_params[np.argmin(rmses)]
print(best_params)


00:08:39 - cmdstanpy - INFO - Chain [1] start processing
00:08:39 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

00:08:40 - cmdstanpy - INFO - Chain [1] start processing
00:08:40 - cmdstanpy - INFO - Chain [1] done processing
00:08:40 - cmdstanpy - ERROR - Chain [1] error: error during processing Unknown error
Optimization terminated abnormally. Falling back to Newton.
00:08:40 - cmdstanpy - INFO - Chain [1] start processing
00:08:40 - cmdstanpy - INFO - Chain [1] done processing
00:08:42 - cmdstanpy - INFO - Chain [1] start processing
00:08:43 - cmdstanpy - INFO - Chain [1] done processing
00:08:43 - cmdstanpy - ERROR - Chain [1] error: error during processing Unknown error
Optimization terminated abnormally. Falling back to Newton.
00:08:43 - cmdstanpy - INFO - Chain [1] start processing
00:08:43 - cmdstanpy - INFO - Chain [1] done processing
00:08:45 - cmdstanpy - INFO - Chain [1] start processing
00:08:45 - cmdstanpy - INFO - Chain [1] done processing
00:08:48 - cmdstanpy - INFO - Chain [1] start processing
00:08:48 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

00:08:48 - cmdstanpy - INFO - Chain [1] start processing
00:08:48 - cmdstanpy - INFO - Chain [1] done processing
00:08:48 - cmdstanpy - ERROR - Chain [1] error: error during processing Unknown error
Optimization terminated abnormally. Falling back to Newton.
00:08:48 - cmdstanpy - INFO - Chain [1] start processing
00:08:48 - cmdstanpy - INFO - Chain [1] done processing
00:08:50 - cmdstanpy - INFO - Chain [1] start processing
00:08:50 - cmdstanpy - INFO - Chain [1] done processing
00:08:50 - cmdstanpy - ERROR - Chain [1] error: error during processing Unknown error
Optimization terminated abnormally. Falling back to Newton.
00:08:50 - cmdstanpy - INFO - Chain [1] start processing
00:08:51 - cmdstanpy - INFO - Chain [1] done processing
00:08:53 - cmdstanpy - INFO - Chain [1] start processing
00:08:53 - cmdstanpy - INFO - Chain [1] done processing
00:08:55 - cmdstanpy - INFO - Chain [1] start processing
00:08:55 - cmdstanpy - INFO - Chain [1] done processing


...

  0%|          | 0/3 [00:00<?, ?it/s]

00:23:38 - cmdstanpy - INFO - Chain [1] start processing
00:23:38 - cmdstanpy - INFO - Chain [1] done processing
00:23:40 - cmdstanpy - INFO - Chain [1] start processing
00:23:40 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:42 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/3 [00:00<?, ?it/s]

00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:46 - cmdstanpy - INFO - Chain [1] start processing
00:23:46 - cmdstanpy - INFO - Chain [1] done processing
00:23:48 - cmdstanpy - INFO - Chain [1] start processing
00:23:48 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Length of values (256) does not match length of index (128)

In [ ]:
print(df_cv)

In [ ]:
print(df_p)

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

In [ ]:
m = Prophet(yearly_seasonality=20)
m.fit(cruiser_df)


In [ ]:
future = m.make_future_dataframe(periods=7)
future[-7:]


In [ ]:
forecast = m.predict(future)
print(forecast[-7:])
weekly_forecast = forecast[-7:]

print(weekly_forecast['yhat'].sum(axis=0))


In [ ]:
fig1 = m.plot(forecast)

,ds
828,2022-07-04
829,2022-07-05
830,2022-07-06
831,2022-07-07
832,2022-07-08
833,2022-07-09
834,2022-07-10
